In [1]:
! pip install -q faiss-cpu
! pip install -q google-generativeai
! pip install -q pi_heif
! pip install -q langchain-community
! pip install -q pdfminer.six
! pip install -q python-dotenv
! pip install -q unstructured
! pip install -q unstructured[local-inference]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━

In [2]:
import gspread
from google.auth import default
from google.colab import drive, auth
drive.mount('/content/drive')

auth.authenticate_user()
credentials, _ = default()
gc = gspread.authorize(credentials)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import csv
import google.generativeai as genai
import gspread
import os
import time
import pandas as pd

from dotenv import load_dotenv
from google.colab import auth
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.base import VectorStoreRetriever
from pathlib import Path
from typing import Generator, Optional
from sklearn.metrics import confusion_matrix

In [4]:
current_folder = os.chdir("/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG")

In [5]:
# Load environmental variables from .env
load_dotenv(dotenv_path="/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/.env")

# Authenticate genai configuration for python session
genai.configure(api_key=("")) # INSERT API Key here

# USE THIS EMBEDDING FUNCTION THROUGHOUT THIS FILE
embedding_function = HuggingFaceEmbeddings(
    model_name=os.getenv("EMBEDDING_MODELS"),
    cache_folder="../models/sentencetransformers"
)

<ipython-input-5-e92dcbbf446f>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
REPORTS_DOCUMENTS_DIR = "/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports"
faiss_dir = "/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/faiss-db"

In [7]:
# Open the spreadsheet by title
sheet = gc.open("Summary Field")

# Get the first worksheet (index 0)
worksheet = sheet.get_worksheet(0)

# Get all values from the worksheet as a list of lists
data = worksheet.get_all_values()

building_data = pd.DataFrame(data[1:], columns=data[0])

In [8]:
for index, building in building_data.iterrows():
    print(REPORTS_DOCUMENTS_DIR+"/CR/"+building["CR_Mapping"]+".pdf")
    print(REPORTS_DOCUMENTS_DIR+"/TR6/"+building["TR6_Mapping"]+".pdf")

/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0006132-1B-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-905406-9B-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-B0004953-1B-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-916628-9C-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0000356-1B-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-905319-9B-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0003902-1A-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-904044-9C-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0003910-1A-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-903721-9A-I1.pdf
/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0004547-1A-I1.pdf
/content/drive/MyDrive/Colab Not

In [9]:
def get_retriever() -> VectorStoreRetriever:
    """
    Loads a Faiss vector store from the local directory and initializes a VectorStoreRetriever.

    The retriever allows you to search the vector store for relevant documents based on embeddings.

    :return: vectorstoreretriever: a retriever object for document retrieval.
    """
    db = FAISS.load_local(faiss_dir, embedding_function,
                          index_name="index", allow_dangerous_deserialization=True)  # Load Faiss index
    retriever = VectorStoreRetriever(vectorstore=db, search_kwargs={"k": 20})  # Retrieve top 20 documents
    return retriever

In [10]:
def generate_response(prompt: str, model: str, temperature: float = 0, second_try: bool = False) -> str:
    """
    Generates a response from a language model for a given prompt.

    :param model:  the large langauge model in use (gemini-pro)
    :param temperature: controls randomness in generation (0 is deterministic)
    :param second_try: whether this is a retry attempt

    :returns: The generated response from the model.
    """

    response = "If this shows, something went wrong while generating a response from google genai."
    if second_try:
        time.sleep(3)  # Wait for 3 seconds before retrying

    try:
        genai.GenerationConfig(temperature=temperature)
        response = genai.GenerativeModel(model_name=model).generate_content(prompt).text  # removed model_name
    except Exception as e:
        print(f"API call failed. Error message: {e}")  # Log error for debugging
        if not second_try:
            print("Retrying in 3 seconds...")
            return generate_response(prompt, model, temperature, second_try=True)  # Recursive retry

    return response

In [11]:
def create_knowledge_base(report_path: str) -> None:
    """
    Creates a knowledge base for a chatbot by processing and indexing PDF reports.

    This function performs the following steps:
    1. Loads PDF reports from the specified paths.
    2. Splits the reports into smaller chunks of text for efficient processing.
    3. Computes embedding vectors for each text chunk using an embedding function.
    4. Builds a FAISS index to store the embeddings for fast similarity search.
    5. Saves the FAISS index to a local directory.

    :param report_paths: A list of file paths to the PDF reports.
    """
    # Initialize docs as an empty list before extending it
    docs = []

    # Load PDF documents
    print(f"Loading {report_path}")  # Log each file being loaded
    loader = UnstructuredPDFLoader(report_path)
    docs.extend(loader.load())

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    docs = text_splitter.split_documents(docs)
    print(f"Created {len(docs)} chunks")  # Log number of chunks

    # Extract text and metadata from documents
    texts = [doc.page_content for doc in docs]
    metadatas = [doc.metadata for doc in docs]

    # Create FAISS index from texts and metadata
    db = FAISS.from_texts(texts, embedding_function, metadatas=metadatas)

    # Save the FAISS index locally
    db.save_local(faiss_dir)  # Assumes environment variable is set

    print(f"FAISS VectorDB has {db.index.ntotal} documents")  # Log number of indexed documents

In [12]:
def generate_genai_response(model: str, gemini_prompt: str, retriever: Optional[VectorStoreRetriever] = None,
                            temperature: float = 1, use_retriever: bool = True) -> str:
    """
    Generates a response to a user query using a knowledge base and a language model.

    This function performs the following steps:

    1. Retrieves relevant documents from the knowledge base based on the query.
    2. Concatenates the content of the relevant documents into a single string.
    3. Constructs a prompt for the language model by combining the query and the relevant documents.
    4. Generates a response using the specified language model and temperature.

    :model: the name of the language model to use (gemini-pro)
    :gemini_prompt: the user's query.
    :retriever: a retriever object to search the knowledge base
    :temperature (float, optional): The temperature parameter, controls randomness
    :use_

    :return: the generated response from the language model.
    """

    if use_retriever:
        # Retrieve the most relevant documents from the knowledge base based on the prompt
        relevant_docs = retriever.get_relevant_documents(prompt, k=10)

        # Combine the content of the relevant documents into a single string
        relevant_docs_str = ""
        for doc in relevant_docs:
            relevant_docs_str += doc.page_content + "\n\n"

        # Create a prompt for the language model by combining the query and the relevant documents
        prompt_full = f"""Answer based on the following context:
            {relevant_docs_str}
            Question: {gemini_prompt}"""
    else:
        prompt_full = f"""Answer based on the following context:
                    {prompt}
                    Question: {gemini_prompt}"""

    # Generate the response using the specified model and temperature
    response = generate_response(prompt_full, model=model, temperature=temperature)

    return response

In [13]:
def remove_db_file():
    os.remove(faiss_dir + "/index.faiss")
    os.remove(faiss_dir + "/index.pkl")

In [14]:
save_dir = "/content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/responses"  # Change to your desired path
os.makedirs(save_dir, exist_ok=True)

In [17]:
for index, building in building_data.iterrows():
    remove_db_file
    CR_FILE = REPORTS_DOCUMENTS_DIR+"/CR/"+building["CR_Mapping"]+".pdf"
    create_knowledge_base(CR_FILE)
    temperature = int(os.getenv("TEMPERATURE"))
    prompt = ("""You are a professional engineer that specializes on Stuctural Engineer.
    Given the following data provide an assessment of the conditions of the building's garage""")

    tries = 0
    while True and tries < 3:
        retriever_cr = get_retriever()
        cr_response = generate_genai_response(
                model=os.getenv("MODEL"), gemini_prompt=prompt, retriever=retriever_cr,
                temperature=temperature
            )
        remove_db_file
        tries+=1
        if len(cr_response) > 750:
            break
        #print(cr_response)

    TR6_FILE = REPORTS_DOCUMENTS_DIR+"/TR6/"+building["TR6_Mapping"]+".pdf"
    time.sleep(10)
    create_knowledge_base(TR6_FILE)
    prompt = ("""You are a professional architect that specializes on Facade inpections.
    Given the following data provide a summary of the conditions of the building""")
    retriever_tr = get_retriever()

    tries = 0
    while True and tries < 3:
        tr6_response = generate_genai_response(
                model=os.getenv("MODEL"), gemini_prompt=prompt, retriever=retriever_tr,
                temperature=temperature
            )
        remove_db_file
        tries+=1
        if len(tr6_response) > 750:
                break
        #print(tr6_response)

    all_response = "Summary of Garage Inspection\n\n" + cr_response + "\n\nSummary of FISP Inspection\n\n" + tr6_response
    print(all_response)
    building_filename = os.path.join(save_dir, f"{building['Address']}_response.txt")

    # Save cr_response to a file
    with open(building_filename, "w") as f:
        f.write(all_response)

Loading /content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/CR/CR-M0006132-1B-I1.pdf
Created 35 chunks
FAISS VectorDB has 35 documents
Loading /content/drive/MyDrive/Colab Notebooks/Multi Agent RAG/reports/TR6/TR6-905406-9B-I1.pdf
Created 41 chunks
FAISS VectorDB has 41 documents
Summary of Garage Inspection

The provided text does not include a condition assessment of the building's garage. Therefore, I am unable to provide an assessment without the necessary data.

Summary of FISP Inspection

The building is in overall good condition. There were no UNSAFE conditions observed at the time of inspection. However, there were some SWARMP (Safe With A Repair and Maintenance Program) conditions observed. These conditions are typically caused by time and exposure to the elements. The building owner is currently undertaking a comprehensive facade repair program to address these conditions. The repairs are expected to be completed in summer of 2023.
Loading /content/drive/MyDrive/Co